In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cnndockbench import home
from cnndockbench.dataloader import loadDatasets
from cnndockbench.featurize import getProteinFeatures, getLigandFeatures
from cnndockbench.utils import getCenters
import os
from tqdm import tqdm
import numpy as np

In [3]:
# get the datasets --> dict
# key = _id
# value = 7 items

# item1 --> systemName
# item2 --> nProteins
# item3 --> nProtocols
# item4 --> ligandFiles
# item5 --> center 
# item6 --> proteinFiles
# item 7 --> expected values --> for each complex data shape (3, 17)
# data shape (nComplexes, 3, nProtocols)
datasets = loadDatasets()


/home/cuzzo/Softwares/Github/htmdmol/htmdmol/readers.py:800: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  parsedtopo.element[idx] = newelem
2019-02-22 18:38:10,892 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:10,897 - htmdmol.molecule - INFO - Removed 14 atoms. 1971 atoms remaining in the molecule.
2019-02-22 18:38:11,132 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:11,133 - htmdmol.molecule - INFO - Removed 20 atoms. 2009 atoms remaining in the molecule.
2019-02-22 18:38:11,257 - htmdmol.molecule - WARNING - Alternative atom locatio

2019-02-22 18:38:15,701 - htmdmol.molecule - INFO - Removed 23 atoms. 1843 atoms remaining in the molecule.
2019-02-22 18:38:15,858 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:15,859 - htmdmol.molecule - INFO - Removed 35 atoms. 1828 atoms remaining in the molecule.
2019-02-22 18:38:16,003 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:16,005 - htmdmol.molecule - INFO - Removed 20 atoms. 3494 atoms remaining in the molecule.
2019-02-22 18:38:16,122 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:16,124 - htmdmol.molecule - INFO - Removed 8 atoms. 1845 atoms remai

2019-02-22 18:38:23,252 - htmdmol.molecule - INFO - Removed 25 atoms. 3811 atoms remaining in the molecule.
2019-02-22 18:38:23,483 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:23,485 - htmdmol.molecule - INFO - Removed 76 atoms. 1984 atoms remaining in the molecule.
2019-02-22 18:38:23,824 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:23,826 - htmdmol.molecule - INFO - Removed 31 atoms. 1886 atoms remaining in the molecule.
2019-02-22 18:38:24,151 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:24,153 - htmdmol.molecule - INFO - Removed 6 atoms. 1726 atoms remai

2019-02-22 18:38:30,138 - htmdmol.molecule - INFO - Removed 6 atoms. 1745 atoms remaining in the molecule.
2019-02-22 18:38:30,384 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:30,386 - htmdmol.molecule - INFO - Removed 6 atoms. 1820 atoms remaining in the molecule.
2019-02-22 18:38:30,508 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:30,509 - htmdmol.molecule - INFO - Removed 3 atoms. 2024 atoms remaining in the molecule.
2019-02-22 18:38:30,630 - htmdmol.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-02-22 18:38:30,631 - htmdmol.molecule - INFO - Removed 11 atoms. 1945 atoms remaini

In [4]:
# generates proteinFeatures and ligFeatures

protFeatures = []
ligFeatures = []
correctValues = []

skipped = []

for _id, value in datasets.items():
    d = value[3:]
    ligs = d[0]
    prots = d[2]
    centers = d[1]
    datas = d[-1]
    for p,c,l,data in tqdm(zip(prots, centers, ligs, datas), total=len(ligs)):
        lFeats = getLigandFeatures(l)
        if lFeats is None:
            skipped.append(l)
            continue
        pFeats = getProteinFeatures(p, c)
        correctValues.append(data)
        protFeatures.append(pFeats)
        ligFeatures.append(lFeats)

protFeatures = np.array(protFeatures)
ligFeatures = np.array(ligFeatures)
correctValues = np.array(correctValues)


    

100%|██████████| 200/200 [00:34<00:00,  5.49it/s]


In [5]:
protFeatures.shape, ligFeatures.shape, correctValues.shape

((199, 24, 24, 24, 7), (199, 1024), (199, 3, 17))

In [6]:

# save arrays
np.save(os.path.join('data', 'protFeats.npy'), protFeatures)
np.save(os.path.join('data', 'ligFeats.npy'), ligFeatures)
np.save(os.path.join('data', 'correctValues.npy'), correctValues)